#Weisfeiler-lehman Kernels 

In [ ]:
#install python-igraph : we use IGraph for manipulating and storing graph data

!pip install python-igraph

In [ ]:
import os
import igraph as ig

In [ ]:
#mounting the data  on drive 

from google.colab import drive
drive.mount('/content/drive')
path = 'data/graph_data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#retreving the file names
path = '/content/drive/MyDrive/data/graph_data/MUTAG'
graph_files = []
for file in os.listdir(path):
  if file.endswith('gml'):
      graph_files.append(os.path.join(path,file))
graph_files.sort()

In [ ]:
#plots the graph 
def plot_graph(G):


In [ ]:
#sample graph
G = ig.read(graph_files[0])
G.vs["id"]
print("Graph : ", end = '\n\n')
print(G)
print(G.vs["id"])
print(G.vs["label"])

Graph : 

IGRAPH U-W- 17 19 --
+ attr: id (v), label (v), weight (e)
+ edges:
2--3 3--4 2--5 6--7 8--9 4--11 11--12 5--12 6--12 7--13 11--14 13--14 8--14
9--15 10--13 10--15 15--16 0--16 1--16
[16.0, 17.0, 1.0, 2.0, 3.0, 6.0, 7.0, 8.0, 11.0, 12.0, 14.0, 4.0, 5.0, 9.0, 10.0, 13.0, 15.0]
['2', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']


In [ ]:
#does label compression 
def f(graph,dict_labels,unique):


  new_labels = {}
  for s_v in unique_labels:

    if s_v in dict_labels.keys():
      new_labels.update({ s_v : dict_labels[s_v] })

    else :
      new_label  = str(len(dict_labels)+1)
      new_labels.update({ s_v  : new_label })    
      dict_labels.update({ s_v : new_label)})




In [ ]:
#does one iteration of WL test for Isomorphism 
def update_compressed_code(graph G):

  
    multiset_g = list()
    for v in G.get_vertices():

      multiset_v = list()

      #multiset-label determination
      for neigh in G[v]:
        multiset_v.append(G[neigh]['label'])

      #sorting each muliset  
      multiset_v.sort() 
      multiset_v.insert(G[v],0)
      multiset_g.append(multiset_v)
      
    unique_multiset_g = list(set(tuple(multiset_g))) #lists are not hashable so convert into tuples

    #label-compression
    compressed_labels = {}




In [ ]:
#1d version of weisfeiler-lehman test for isomorphism 
def weisfeiler_lehman_test(graph G1,graph G2):

  V = G1.V
  for i in range(V):

    



Example for testing isomorphism of graphs 

In [ ]:
#weisfeiler lehman subtree kernel
def weisfeiler_lehman_subtree_kernel():


In [ ]:

def weisfeiler_lehman_edge_kernel():
